## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.

1\. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

2\. Find out the value of 'x'

3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [58]:
import pandas as pd
import numpy as np
import numpy.random as npr
npr.seed(123)
import datetime as dt
from math import fabs

### 1
Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

In [2]:
#set filename
file_name='/Users/annabraghetto/data/data_000637.txt'
#set N
N=npr.randint(10000,1310720)
print('Number of rows: ', N)
#read the file
data=pd.read_csv(file_name, nrows=N)

data

Number of rows:  783630


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


### 2
Find out the value of 'x'

In [76]:
#compute x
i=0
j=0
while j < N:
    x=0
    h=0
    l=i
    while data.loc[i,'ORBIT_CNT'] == data.loc[i+1,'ORBIT_CNT']:
        if data.loc[i+1,'BX_COUNTER'] -data.loc[i,'BX_COUNTER'] >0:
            x += data.loc[i+1,'BX_COUNTER'] - data.loc[i,'BX_COUNTER']
        i += 1
        h += 1
    print(data.loc[i,'ORBIT_CNT'],data.loc[i+1,'ORBIT_CNT'], x+data.loc[l,'BX_COUNTER'], h)
    i +=1
    #print('x =', x, 'BX_COUNTER' )
    j = i
    



3869200167 3869200168 3237 42
3869200168 3869200169 3626 84
3869200169 3869200170 2926 126
3869200170 3869200171 3479 97
3869200171 3869200172 3677 108
3869200172 3869200173 3586 88
3869200173 3869200174 3325 87
3869200174 3869200175 3113 127
3869200175 3869200176 3823 127
3869200176 3869200177 3540 50
3869200177 3869200178 3383 110
3869200178 3869200179 1504 93
3869200179 3869200180 2337 127
3869200180 3869200181 3630 84
3869200181 3869200182 3659 117
3869200182 3869200183 3564 57
3869200183 3869200184 2466 122
3869200184 3869200185 3550 61
3869200185 3869200186 3472 134
3869200186 3869200187 3565 94
3869200187 3869200188 3067 91
3869200188 3869200189 2373 127
3869200189 3869200190 3418 37
3869200190 3869200191 3667 85
3869200191 3869200192 3663 137
3869200192 3869200193 3599 66
3869200193 3869200194 3685 146
3869200194 3869200195 2884 163
3869200195 3869200196 3652 75
3869200196 3869200195 6 1
3869200195 3869200196 3563 0
3869200196 3869200197 3581 56
3869200197 3869200198 2229 119
3

3869200422 3869200423 3533 127
3869200423 3869200424 3512 72
3869200424 3869200425 3439 92
3869200425 3869200426 3362 80
3869200426 3869200427 3440 123
3869200427 3869200428 3713 163
3869200428 3869200429 3092 28
3869200429 3869200430 3661 103
3869200430 3869200431 3433 122
3869200431 3869200432 3510 121
3869200432 3869200433 3621 92
3869200433 3869200434 3395 56
3869200434 3869200435 3435 97
3869200435 3869200436 3369 114
3869200436 3869200437 3720 146
3869200437 3869200438 3284 71
3869200438 3869200439 3429 103
3869200439 3869200440 3093 87
3869200440 3869200441 3638 77
3869200441 3869200442 3638 100
3869200442 3869200443 3766 171
3869200443 3869200444 3589 53
3869200444 3869200445 3042 106
3869200445 3869200446 3500 66
3869200446 3869200447 3652 121
3869200447 3869200448 3775 121
3869200448 3869200449 1376 72
3869200449 3869200450 3527 97
3869200450 3869200451 3585 106
3869200451 3869200452 3442 93
3869200452 3869200453 3331 88
3869200453 3869200454 3067 84
3869200454 3869200455 375

3869200723 3869200724 3686 150
3869200724 3869200725 874 50
3869200725 3869200726 2647 127
3869200726 3869200727 3582 99
3869200727 3869200728 3406 74
3869200728 3869200729 3701 84
3869200729 3869200730 2968 123
3869200730 3869200731 3609 120
3869200731 3869200732 3611 65
3869200732 3869200733 3198 111
3869200733 3869200734 3355 87
3869200734 3869200735 2884 124
3869200735 3869200736 3669 125
3869200736 3869200737 3605 82
3869200737 3869200738 3530 113
3869200738 3869200739 1174 60
3869200739 3869200740 2633 127
3869200740 3869200741 3349 127
3869200741 3869200742 3448 39
3869200742 3869200743 3648 104
3869200743 3869200744 3259 63
3869200744 3869200745 3424 121
3869200745 3869200746 3613 125
3869200746 3869200747 3336 62
3869200747 3869200748 3640 109
3869200748 3869200749 3618 77
3869200749 3869200750 3154 187
3869200750 3869200751 3428 64
3869200751 3869200752 3626 84
3869200752 3869200753 3505 98
3869200753 3869200754 3671 120
3869200754 3869200755 3574 103
3869200755 3869200756 31

3869201044 3869201045 3290 142
3869201045 3869201046 3532 52
3869201046 3869201047 3288 120
3869201047 3869201048 3595 110
3869201048 3869201049 3445 138
3869201049 3869201050 3576 67
3869201050 3869201051 3520 93
3869201051 3869201052 2712 100
3869201052 3869201053 3250 109
3869201053 3869201054 3635 79
3869201054 3869201055 2965 36
3869201055 3869201056 3467 101
3869201056 3869201057 3585 100
3869201057 3869201058 3774 201
3869201058 3869201059 3535 69
3869201059 3869201060 3270 42
3869201060 3869201061 3412 114
3869201061 3869201062 3557 127
3869201062 3869201063 3429 152
3869201063 3869201064 3503 97
3869201064 3869201065 3537 59
3869201065 3869201066 3036 108
3869201066 3869201067 3698 112
3869201067 3869201068 3611 86
3869201068 3869201067 3 0
3869201067 3869201068 3557 0
3869201068 3869201067 7 1
3869201067 3869201068 3563 0
3869201068 3869201069 3631 129
3869201069 3869201070 3454 60
3869201070 3869201071 2666 115
3869201071 3869201072 3436 127
3869201072 3869201073 3696 119
38

3869201285 3869201286 950 56
3869201286 3869201287 2532 127
3869201287 3869201288 3709 127
3869201288 3869201289 3722 84
3869201289 3869201290 3512 63
3869201290 3869201291 1886 106
3869201291 3869201292 2978 127
3869201292 3869201293 3640 100
3869201293 3869201294 3519 84
3869201294 3869201295 3490 74
3869201295 3869201296 3280 76
3869201296 3869201297 3420 84
3869201297 3869201298 3538 119
3869201298 3869201299 3653 138
3869201299 3869201300 3728 125
3869201300 3869201301 1852 87
3869201301 3869201302 3411 98
3869201302 3869201303 3808 140
3869201303 3869201304 3434 53
3869201304 3869201305 2385 89
3869201305 3869201306 3500 127
3869201306 3869201307 3664 93
3869201307 3869201308 3578 109
3869201308 3869201309 3284 76
3869201309 3869201310 3463 102
3869201310 3869201311 3143 127
3869201311 3869201312 3661 107
3869201312 3869201313 3619 84
3869201313 3869201314 1832 62
3869201314 3869201315 1917 127
3869201315 3869201316 3385 105
3869201316 3869201317 3468 102
3869201317 3869201318 36

...

3869206367 3869206368 3524 127
3869206368 3869206369 3585 79
3869206369 3869206370 3870 72
3869206370 3869206371 2791 102
3869206371 3869206372 3600 78
3869206372 3869206373 3628 86
3869206373 3869206374 3508 44
3869206374 3869206375 3705 146
3869206375 3869206376 3690 147
3869206376 3869206377 3467 25
3869206377 3869206378 3432 113
3869206378 3869206379 3675 108
3869206379 3869206380 3308 60
3869206380 3869206381 3677 117
3869206381 3869206382 3558 120
3869206382 3869206383 3644 113
3869206383 3869206384 3433 110
3869206384 3869206385 3790 126
3869206385 3869206386 3698 97
3869206386 3869206387 3473 59
3869206387 3869206388 3457 65
3869206388 3869206389 3757 158
3869206389 3869206390 3466 77
3869206390 3869206391 3410 70
3869206391 3869206392 3532 87
3869206392 3869206393 3365 77
3869206393 3869206394 3044 58
3869206394 3869206395 3655 170
3869206395 3869206396 3082 96
3869206396 3869206397 3577 40
3869206397 3869206398 3560 126
3869206398 3869206399 3537 141
3869206399 3869206400 326

3869206658 3869206659 3951 252
3869206659 3869206660 3628 165
3869206660 3869206661 3620 157
3869206661 3869206662 3625 173
3869206662 3869206663 3383 192
3869206663 3869206664 3673 160
3869206664 3869206665 3830 148
3869206665 3869206666 3684 138
3869206666 3869206667 3511 100
3869206667 3869206668 3755 180
3869206668 3869206669 3780 228
3869206669 3869206670 3930 251
3869206670 3869206671 3575 153
3869206671 3869206672 3495 154
3869206672 3869206673 3504 117
3869206673 3869206674 3718 198
3869206674 3869206675 3600 123
3869206675 3869206676 3590 111
3869206676 3869206677 3677 125
3869206677 3869206678 3459 105
3869206678 3869206679 3801 206
3869206679 3869206680 3665 195
3869206680 3869206681 3684 151
3869206681 3869206682 3620 151
3869206682 3869206683 3749 145
3869206683 3869206684 3598 120
3869206684 3869206685 3620 114
3869206685 3869206684 1 0
3869206684 3869206685 3561 2
3869206685 3869206684 6 0
3869206684 3869206685 3560 0
3869206685 3869206684 4 3
3869206684 3869206685 3563 

3869206937 3869206938 3507 88
3869206938 3869206939 3569 174
3869206939 3869206940 3660 138
3869206940 3869206941 3561 160
3869206941 3869206942 3765 208
3869206942 3869206943 3693 144
3869206943 3869206944 3585 150
3869206944 3869206945 3666 157
3869206945 3869206946 3305 192
3869206946 3869206947 3824 150
3869206947 3869206946 0 0
3869206946 3869206947 3561 0
3869206947 3869206948 3846 194
3869206948 3869206949 3529 55
3869206949 3869206950 3614 95
3869206950 3869206951 3629 117
3869206951 3869206952 3327 115
3869206952 3869206953 3662 141
3869206953 3869206954 3633 141
3869206954 3869206955 3521 110
3869206955 3869206956 3677 156
3869206956 3869206957 3672 117
3869206957 3869206956 1 0
3869206956 3869206957 3563 1
3869206957 3869206958 3644 163
3869206958 3869206959 3726 163
3869206959 3869206960 3903 190
3869206960 3869206961 3708 136
3869206961 3869206962 3668 116
3869206962 3869206963 3686 220
3869206963 3869206964 3616 83
3869206964 3869206965 3629 131
3869206965 3869206966 3715

3869207285 3869207286 3781 197
3869207286 3869207287 3549 100
3869207287 3869207288 3693 134
3869207288 3869207289 3597 131
3869207289 3869207290 3707 138
3869207290 3869207291 3569 106
3869207291 3869207292 3736 166
3869207292 3869207293 3274 94
3869207293 3869207294 3532 112
3869207294 3869207295 3325 161
3869207295 3869207296 3259 98
3869207296 3869207297 3410 126
3869207297 3869207296 1 1
3869207296 3869207297 3561 1
3869207297 3869207298 3732 106
3869207298 3869207297 3 1
3869207297 3869207298 3563 0
3869207298 3869207299 3580 95
3869207299 3869207300 3533 107
3869207300 3869207301 3605 67
3869207301 3869207302 3571 68
3869207302 3869207303 3472 52
3869207303 3869207304 3005 94
3869207304 3869207305 3589 131
3869207305 3869207306 3508 91
3869207306 3869207307 3030 92
3869207307 3869207308 3584 117
3869207308 3869207309 3568 60
3869207309 3869207310 3298 103
3869207310 3869207311 3555 92
3869207311 3869207312 3675 109
3869207312 3869207313 3168 90
3869207313 3869207314 3425 126
386

KeyError: 'the label [783630] is not in the [index]'

168
